# Wordle solver
We need to make some assumptions to write an explicit solution, such as:

* Words are likely to be the solution proportional to their use in the corpus
  * In reality, there may be a non-linear relationship, e.g. square root
  * **My intuition is to use metric `letter_freq += 1 + math.sqrt(word_frequency)`** for each occurence of letter
* Based on the above, letters are likely to be in the solution proportional to their (weighted) use in the corpus
* Getting a "green" is more valuable than getting a "yellow" (take letter frequency at individual positions into account)
* etc.

A better solution may be to use reinforcement learning, so as to avoid making these assumptions.

In [93]:
import pandas as pd
import numpy as np
import string
from collections import defaultdict
import math

In [216]:
print('Preparing vocabulary...')

Preparing vocabulary...


In [217]:
# https://www.kaggle.com/datasets/rtatman/english-word-frequency?resource=download
words = pd.read_csv('./data/unigram_freq.csv')
az = string.ascii_lowercase

In [218]:
# Filter to 5 letter words
words = words[words['word'].str.len() == 5]
words['freq'] = words['count'] / words['count'].max()
print(len(words))
words.head()

39933


,word,count,freq
35,about,1226734006,1.000000
45,other,978481319,0.797631
56,which,810514085,0.660709
57,their,782849411,0.638157
62,there,701170205,0.571575


In [219]:
# Allowable wordle words
# https://github.com/tabatkins/wordle-list
allowed_words = pd.read_csv('./data/wordle-allowed-words.txt', header=None)
print(len(allowed_words))
allowed_words.head()

12947


,0
0,women
1,nikau
2,swack
3,feens
4,fyles


In [220]:
# Load Wordle solutions (test set)
# https://medium.com/@owenyin/here-lies-wordle-2021-2027-full-answer-list-52017ee99e86
solutions = pd.read_csv('./data/wordle_solutions.txt', sep=' ', header=None)
solutions = list(solutions[5].str.lower())
# Filter out the "Don't Repost" watermark
solutions = [s for s in solutions if s != 'dont' and s != 'repost']
print(len(solutions))
# Make sure solutions are in allowed words
print(np.array([solution in allowed_words[0].values for solution in solutions]).sum() / len(solutions))
[solution for solution in solutions if solution not in allowed_words[0].values]

2315
0.9974082073434125


['agora', 'pupal', 'lynch', 'fibre', 'slave', 'wench']

In [221]:
# https://static.nytimes.com/newsgraphics/2022/01/25/wordle-solver/assets/solutions.txt
solutions2 = pd.read_csv('./data/nytimes-solutions.txt', header=None)[0].values
print(len(solutions2))
print(np.array([solution in allowed_words[0].values for solution in solutions2]).sum() / len(solutions2))
[solution for solution in solutions2 if solution not in allowed_words[0].values]

2309
1.0


[]

In [222]:
set(solutions).difference(set(solutions2))

{'agora', 'fibre', 'lynch', 'pupal', 'slave', 'wench'}

In [223]:
# Add the 
allowed_words_set = set(allowed_words[0])
allowed_words_set = allowed_words_set.union(set(solutions).difference(set(solutions2)))

# Filter word frequency table to include allowed words only
words = words[words['word'].isin(allowed_words_set)]
len(words)

8078

In [224]:
# Add the other allowed words but with count and freq 0
counted_words = set(words['word'])
new_rows = []
for w in allowed_words[0]:
    if w not in counted_words:
        new_row = {'word': w, 'count':0, 'freq':0}
        new_rows.append(new_row)

words = words.append(pd.DataFrame(new_rows))

## 1. Find "best" words

In [225]:
# Letter frequency
letter_freq = defaultdict(int)
for _, w in words.iterrows():
    for char in w['word']:
#         Many ways to weight this:
#         letter_freq[char] += w['freq']
#         letter_freq[char] += 1
        letter_freq[char] += 1 + math.sqrt(w['freq'])

sorted_letters = sorted(letter_freq.items(), key=lambda x: -x[1])
sorted_letters[:5]

[('e', 6829.547059012467),
 ('s', 6798.21380777582),
 ('a', 6119.704131441435),
 ('o', 4537.117539948098),
 ('r', 4268.127371788989)]

In [226]:
# Find the best "starting words"
# Best: maximizing total letter_freq (but 0 score for repeated letter)
# This doesn't take into account letter position
word_goodness = defaultdict(float)
for w in allowed_words[0]:
    used_chars = set()
    for char in w:
        if char in used_chars:
            continue
        word_goodness[w] += letter_freq[char]
        used_chars.add(char)
        
sorted_words = sorted(word_goodness.items(), key=lambda x: -x[1])
sorted_words[:5]

[('aeros', 28552.709909966812),
 ('arose', 28552.70990996681),
 ('soare', 28552.70990996681),
 ('raise', 27859.327260501737),
 ('arise', 27859.327260501737)]

In [227]:
# Take letter position into account
# Letter frequency per position
letter_pos_freq = [defaultdict(float) for _ in range(5)]
for _, w in words.iterrows():
    for i, char in enumerate(w['word']):
#         Many ways to weight this:
#         letter_freq[char] += w['freq']
#         letter_freq[char] += 1
        letter_pos_freq[i][char] += 1 + math.sqrt(w['freq'])

sorted_pos_letters = [sorted(letter_pos_freq[i].items(), key=lambda x: -x[1]) for i in range(5)]
[s[0] for s in sorted_pos_letters]

[('s', 1600.5164740570062),
 ('a', 2308.278555858041),
 ('a', 1273.7456442333491),
 ('e', 2382.654624241403),
 ('s', 4028.057199519846)]

In [228]:
# Find the best "starting words"
# Best: maximizing total letter_freq (but 0 score for repeated letter)
# This doesn't take into account letter position
word_goodness_pos = defaultdict(float)
for w in allowed_words[0]:
    used_chars = set()
    for i, char in enumerate(w):
#         if char in used_chars:
#             continue
        word_goodness_pos[w] += letter_pos_freq[i][char]
        used_chars.add(char)
        
sorted_words_pos = sorted(word_goodness_pos.items(), key=lambda x: -x[1])
sorted_words_pos[:5]

[('sores', 11379.756926125297),
 ('sanes', 11305.615343122061),
 ('sales', 11185.506599271926),
 ('sones', 11142.872085069745),
 ('soles', 11022.76334121961)]

In [229]:
word_good_df = pd.DataFrame(dict(word_goodness), index=['']).T
word_good_df.columns = ['overall']

In [230]:
word_good_df_pos = pd.DataFrame(dict(word_goodness_pos), index=['']).T
word_good_df_pos.columns = ['position']

In [231]:
word_good_df_all = word_good_df.join(word_good_df_pos)
word_good_df_all['avg'] = (
    (word_good_df_all['overall'] / word_good_df_all['overall'].max())
    + (word_good_df_all['position'] / word_good_df_all['position'].max())
) / 2
best_words = word_good_df_all.sort_values(['avg'], ascending=False)
best_words.head()

,overall,position,avg
tares,27407.366733,10781.425154,0.953654
lares,27474.172506,10533.574717,0.943934
cares,26095.515636,10887.691356,0.935351
pares,26074.256873,10821.062589,0.932051
dares,26521.862604,10636.135606,0.931764


## 2. Solver (hard mode)
This will always use existing clues. Note that this is not an optimal strategy for playing not in hard mode - where using a completely different set of letters is preferred for more information gain.

We can define a set of allowable letters for each position, which is initially all letters, then narrows.

Start by doing an example.

In [232]:
best_word_list = best_words.index
best_word_array = np.array([np.array([c for c in w]) for w in best_word_list])

def use_clues(word, result):
    # Special case: Same letter is green and grey
    # Have to keep track of position of green chars
    green_idx = defaultdict(list)
    grey = set()
    for i, r in enumerate(result):
        char = word[i]
        if r == 'grey':
            grey.add(char)
        elif r == 'green':
            green_idx[char].append(i)
    green_and_grey = set(green_idx.keys()).intersection(grey)
    for i, r in enumerate(result):
        char = word[i]
        if r == 'green':
            a[i] = set([char])
        elif r == 'yellow':
            try:
                a[i].remove(char)
            except:
                continue
            b.add(char)
        elif r == 'grey':
            if char in green_and_grey:
                # Remove this option from all but the green positions
                for i, pos in enumerate(a):
                    if i in green_idx[char]:
                        continue
                    try:
                        pos.remove(char)
                    except:
                        continue
            else:
                # Remove from all positions
                for pos in a:
                    try:
                        pos.remove(char)
                    except:
                        continue

# Find all allowable words matching the constraints
def find_ok_words(a, b):
    a_met = (
        np.in1d(best_word_array[:,0], list(a[0]))
        & np.in1d(best_word_array[:,1], list(a[1]))
        & np.in1d(best_word_array[:,2], list(a[2]))
        & np.in1d(best_word_array[:,3], list(a[3]))
        & np.in1d(best_word_array[:,4], list(a[4]))
    )
#     print('meets a:', a_met.sum())
    if len(b) > 0:
        b_met_each = []
        for char in b:
            b_met_each.append(np.array([char in word for word in best_word_list]))
        b_met_all = np.array([True] * len(best_word_list))
        for b_met in b_met_each:
            b_met_all = b_met_all & b_met
    else:
        b_met_all = np.array([True] * len(best_word_list))
#     print('meets b:', b_met_all.sum())
    ok_words = best_word_array[a_met & b_met_all]
    return [''.join(w) for w in ok_words]

# Find the best clue word (most information)
def find_clue(a, b):
    ok_words = find_ok_words(a, b)
    # ok_words is already ranked by how useful the letters are
    return ok_words[0]

# Find the most likely solution word (most common)
def find_solution(a, b):
    ok_words = find_ok_words(a, b)
    print(len(ok_words), 'matching words.')
    # Get the most common word in ok_words
    ok_words_sorted = words[words['word'].isin(ok_words)]
    return ok_words_sorted['word'].values[0]

def find_clues_solutions(a, b, guess_num, verbose=False):
    ok_words = find_ok_words(a, b)
    
    print()
    print(len(ok_words), 'valid words.')
    
    # Get the most common word in ok_words
    most_common_words = words[words['word'].isin(ok_words)]
    most_common_words.index = most_common_words['word']
    most_common_words = most_common_words[['freq']]
    most_common_words.columns = ['Word Frequency']
    most_common_words.index.name = None
        
    best_info_words = best_words.loc[ok_words][['avg']]
    best_info_words.columns = ['Information Gain']
        
    best_of_both = best_info_words.join(most_common_words)
    
#     if guess_num < 2:
#         print('\nRecommended word (best information gain):')
#         print(best_info_words.index[0])
#     elif guess_num < 5:
#         print('\nRecommended word (best combination of information gain and word frequency):')
#         print(best_of_both.index[0])
#     else:
#         print('\nRecommended word (most common word matching constraints):')
#         print(most_common_words.index[0])

    # New way: take a weighted average of the rank of each
    best_of_both['word'] = best_of_both.index
    best_of_both = best_of_both.sort_values(['Information Gain']).reset_index().drop(['index'],axis=1)
    best_of_both['info_idx'] = best_of_both.index
    best_of_both = best_of_both.sort_values(['Word Frequency']).reset_index().drop(['index'],axis=1)
    best_of_both['freq_idx'] = best_of_both.index
    best_of_both.index = best_of_both['word']

    freq_weight = 0.2 * guess_num
    info_weight = 1 - freq_weight
    
    best_of_both['Combined Score'] = best_of_both['info_idx']*info_weight + best_of_both['freq_idx']*freq_weight
    best_of_both = best_of_both.sort_values(['Combined Score'], ascending=False)
    
    if verbose == True:
        print('\nFrequent letters in common positions):')
        print(best_info_words.head())
        print('\nMost common words:')
        print(most_common_words.head())
        print(f'\nWeighted combination (w_info={info_weight}, w_freq={freq_weight}):')
        print(best_of_both[['Combined Score', 'Information Gain', 'Word Frequency']].head(5))
        
    print('\nRecommended word:')
    print(best_of_both.index[0])
    print()
    return best_of_both.index[0]
    
# Shorthands to make entering results easier
sh = ['grey', 'yellow', 'green']

In [234]:
verbose = False
auto_mode = False

while True:
    # Allowed letters
    a = [set(az) for _ in range(5)]
    # Necessary letters
    b = set()
    
    quit = False
    print('\n=============\n')
    print('\nWORDLE SOLVER')
    word = find_clues_solutions(a, b, 0, verbose=verbose)
    print()
    for i in range(1, 7):
        if not auto_mode:
            word = input(f'Guess {i} (or type "exit", "restart"): ')
        else:
            print(f"Enter the word ({word}) in Wordle now.")
        if word == 'restart':
            break
        elif word == 'exit':
            quit = True
            break
        result = input('Result? (0=grey, 1=yellow, 2=green): ')
        result = [sh[int(s)] for s in result]
        use_clues(word, result)
#         print('Best solution word is:', find_solution(a, b))
#         print('Best clue word is:', find_clue(a,b))
        try:
            word = find_clues_solutions(a, b, i, verbose=verbose)
        except:
            print('Impossible! Restarting.')
            break
        print('')
        
    if quit:
        break




WORDLE SOLVER

12947 valid words.

Recommended word:
tares


Guess 1 (or type "exit", "restart"): exit


## Test the solver against Wordle solutions

In [235]:
def test_guess(guess, solution):
    assert len(solution) == 5
    assert len(guess) == 5
    response = ''
    for i in range(5):
        # Green
        if guess[i] == solution[i]:
            response += '2'
        elif guess[i] in solution:
            response += '1'
        else:
            response += '0'
    print(response)
    return [sh[int(s)] for s in response]
    
def test_solver(solution, a, b, verbose=False):
    assert len(solution) == 5
    guess = None
    i = 0
    while i < 20:
        i += 1
        guess = find_clues_solutions(a, b, i-1, verbose=verbose)
        if guess == solution:
            print(f'Solved in {i} guesses.')
            return i
        result = test_guess(guess, solution)
        use_clues(guess, result)
    return 20

In [ ]:
solution_test_results = dict()
for solution in solutions:
    a = [set(az) for _ in range(5)]
    b = set()
    try:
        solution_test_results[solution] = test_solver(solution, a, b)
    except:
        solution_test_results[solution] = 0


12947 valid words.

Recommended word:
tares

01100

245 valid words.

Recommended word:
board

00110

36 valid words.

Recommended word:
pilar

02022

4 valid words.

Recommended word:
cigar

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

10110

54 valid words.

Recommended word:
route

20111

2 valid words.

Recommended word:
recut

22022

1 valid words.

Recommended word:
rebut

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

00001

294 valid words.

Recommended word:
solid

20010

26 valid words.

Recommended word:
shiny

20102

3 valid words.

Recommended word:
sissy

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

00000

856 valid words.

Recommended word:
could

00100

67 valid words.

Recommended word:
bunny

02000

8 valid words.

Recommended word:
gummi

02210

1 valid words.

Recommended word:
humph

Solved in 5 guesses.

12947 valid words.

Recommended word:
tares

01010

271 valid words.

Recommended word:
alone

20

Recommended word:
whelp

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

21100

29 valid words.

Recommended word:
tolar

20111

2 valid words.

Recommended word:
trail

22202

1 valid words.

Recommended word:
trawl

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

10000

261 valid words.

Recommended word:
point

01001

9 valid words.

Recommended word:
cloth

00110

3 valid words.

Recommended word:
outdo

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

01010

271 valid words.

Recommended word:
alone

20202

8 valid words.

Recommended word:
abode

21212

1 valid words.

Recommended word:
adobe

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

01100

245 valid words.

Recommended word:
board

00210

47 valid words.

Recommended word:
chair

20201

7 valid words.

Recommended word:
crazy

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

00121

51 valid words.

Recommended word:
sober

22022




12947 valid words.

Recommended word:
tares

20020

41 valid words.

Recommended word:
toney

22020

11 valid words.

Recommended word:
toped

22020

2 valid words.

Recommended word:
towel

22020

1 valid words.

Recommended word:
totem

Solved in 5 guesses.

12947 valid words.

Recommended word:
tares

00120

282 valid words.

Recommended word:
riley

10020

105 valid words.

Recommended word:
poker

02022

27 valid words.

Recommended word:
homer

02022

17 valid words.

Recommended word:
cover

02022

11 valid words.

Recommended word:
boxer

02022

8 valid words.

Recommended word:
dozer

02022

4 valid words.

Recommended word:
goner

02022

2 valid words.

Recommended word:
fouer

02022

1 valid words.

Recommended word:
wooer

Solved in 10 guesses.

12947 valid words.

Recommended word:
tares

10100

64 valid words.

Recommended word:
court

01012

6 valid words.

Recommended word:
droit

02202

2 valid words.

Recommended word:
front

Solved in 4 guesses.

12947 valid words.



Recommended word:
tares

00120

282 valid words.

Recommended word:
riley

10021

16 valid words.

Recommended word:
buyer

00122

3 valid words.

Recommended word:
yoker

10022

2 valid words.

Recommended word:
hyper

Solved in 5 guesses.

12947 valid words.

Recommended word:
tares

20000

80 valid words.

Recommended word:
tonic

20000

9 valid words.

Recommended word:
tummy

21120

2 valid words.

Recommended word:
thumb

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

00100

259 valid words.

Recommended word:
round

12001

3 valid words.

Recommended word:
dolor

22011

2 valid words.

Recommended word:
dobro

22021

1 valid words.

Recommended word:
dowry

Solved in 5 guesses.

12947 valid words.

Recommended word:
tares

10000

261 valid words.

Recommended word:
point

01002

12 valid words.

Recommended word:
clout

00112

2 valid words.

Recommended word:
ought

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

00010

636 valid words.




Recommended word:
tares

00221

20 valid words.

Recommended word:
sorel

20220

15 valid words.

Recommended word:
siren

20220

11 valid words.

Recommended word:
shred

20220

6 valid words.

Recommended word:
spree

20221

3 valid words.

Recommended word:
screw

20220

2 valid words.

Recommended word:
surer

Solved in 7 guesses.

12947 valid words.

Recommended word:
tares

02100

92 valid words.

Recommended word:
dairy

12110

7 valid words.

Recommended word:
radio

Solved in 3 guesses.

12947 valid words.

Recommended word:
tares

00110

241 valid words.

Recommended word:
ronde

22002

7 valid words.

Recommended word:
roule

22202

1 valid words.

Recommended word:
rouge

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

00210

100 valid words.

Recommended word:
borne

00201

29 valid words.

Recommended word:
peril

22200

6 valid words.

Recommended word:
perry

22210

2 valid words.

Recommended word:
perch

Solved in 5 guesses.

12947 valid words.

Re

...


Recommended word:
tares

00110

241 valid words.

Recommended word:
ronde

10001

25 valid words.

Recommended word:
fiery

Solved in 3 guesses.

12947 valid words.

Recommended word:
tares

01010

271 valid words.

Recommended word:
alone

10001

41 valid words.

Recommended word:
decay

12010

1 valid words.

Recommended word:
media

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

20100

34 valid words.

Recommended word:
truly

22200

5 valid words.

Recommended word:
trunk

Solved in 3 guesses.

12947 valid words.

Recommended word:
tares

11011

32 valid words.

Recommended word:
slate

10211

4 valid words.

Recommended word:
beast

02222

3 valid words.

Recommended word:
yeast

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

01110

154 valid words.

Recommended word:
relay

11120

4 valid words.

Recommended word:
clear

Solved in 3 guesses.

12947 valid words.

Recommended word:
tares

00001

294 valid words.

Recommended word:
solid

200


12947 valid words.

Recommended word:
tares

01100

245 valid words.

Recommended word:
board

00210

47 valid words.

Recommended word:
chair

Solved in 3 guesses.

12947 valid words.

Recommended word:
tares

10010

160 valid words.

Recommended word:
monte

01011

8 valid words.

Recommended word:
depot

22021

1 valid words.

Recommended word:
detox

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

00221

20 valid words.

Recommended word:
sorel

20220

15 valid words.

Recommended word:
siren

20220

11 valid words.

Recommended word:
shred

22220

1 valid words.

Recommended word:
shrew

Solved in 5 guesses.

12947 valid words.

Recommended word:
tares

20011

10 valid words.

Recommended word:
toise

20022

3 valid words.

Recommended word:
tense

Solved in 3 guesses.

12947 valid words.

Recommended word:
tares

00020

419 valid words.

Recommended word:
money

00020

130 valid words.

Recommended word:
filed

01122

2 valid words.

Recommended word:
plied

S


Recommended word:
ronde

10102

6 valid words.

Recommended word:
brine

02222

2 valid words.

Recommended word:
urine

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

02001

108 valid words.

Recommended word:
salon

12000

28 valid words.

Recommended word:
daisy

02022

5 valid words.

Recommended word:
massy

02222

2 valid words.

Recommended word:
gassy

Solved in 5 guesses.

12947 valid words.

Recommended word:
tares

21100

29 valid words.

Recommended word:
tolar

20121

1 valid words.

Recommended word:
trial

Solved in 3 guesses.

12947 valid words.

Recommended word:
tares

01100

245 valid words.

Recommended word:
board

01120

6 valid words.

Recommended word:
flora

Solved in 3 guesses.

12947 valid words.

Recommended word:
tares

00120

282 valid words.

Recommended word:
riley

10020

105 valid words.

Recommended word:
poker

10022

3 valid words.

Recommended word:
duper

01222

1 valid words.

Recommended word:
upper

Solved in 5 guesses.

12


12947 valid words.

Recommended word:
tares

00000

856 valid words.

Recommended word:
could

00001

17 valid words.

Recommended word:
dinky

12202

1 valid words.

Recommended word:
windy

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

10001

143 valid words.

Recommended word:
south

10120

7 valid words.

Recommended word:
lusty

02222

5 valid words.

Recommended word:
busty

02222

4 valid words.

Recommended word:
dusty

Solved in 5 guesses.

12947 valid words.

Recommended word:
tares

02001

108 valid words.

Recommended word:
salon

22200

8 valid words.

Recommended word:
sally

Solved in 3 guesses.

12947 valid words.

Recommended word:
tares

00000

856 valid words.

Recommended word:
could

01010

29 valid words.

Recommended word:
lingo

11012

1 valid words.

Recommended word:
igloo

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

00210

100 valid words.

Recommended word:
borne

00211

3 valid words.

Recommended word:
nerdy

So


12947 valid words.

Recommended word:
tares

10012

92 valid words.

Recommended word:
poets

01112

1 valid words.

Recommended word:
ethos

Solved in 3 guesses.

12947 valid words.

Recommended word:
tares

00110

241 valid words.

Recommended word:
ronde

10022

7 valid words.

Recommended word:
pride

02222

2 valid words.

Recommended word:
bride

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

10010

160 valid words.

Recommended word:
monte

01011

8 valid words.

Recommended word:
depot

01021

1 valid words.

Recommended word:
extol

Solved in 4 guesses.

12947 valid words.

Recommended word:
tares

00110

241 valid words.

Recommended word:
ronde

11011

2 valid words.

Recommended word:
decor

Solved in 3 guesses.

12947 valid words.

Recommended word:
tares

00000

856 valid words.

Recommended word:
could

02000

78 valid words.

Recommended word:
boing

12000

6 valid words.

Recommended word:
hobby

Solved in 4 guesses.

12947 valid words.

Recommende

In [ ]:
df = pd.DataFrame(solution_test_results, index=['guesses']).T
df.plot.hist(bins=df['guesses'].max()+1)

In [ ]:
df.query('guesses == 0')

In [179]:
print(len(df))
print(1 - len(df.query('(guesses > 6) | (guesses == 0)')) / len(df))
df.query('guesses > 6')

2317
0.9771255934397929


,guesses
wooer,10
goner,8
hatch,7
surer,7
ferry,7
swill,7
gawky,7
sever,7
stalk,7
cramp,7


In [176]:
a = [set(az) for _ in range(5)]
b = set()
test_solver('hound', a, b, verbose=True)


12947 valid words.

Frequent letters in common positions):
       Information Gain
tares          0.953709
lares          0.943887
cares          0.935361
pares          0.932017
dares          0.931812

Most common words:
       Word Frequency
about        1.000000
other        0.797631
which        0.660709
their        0.638157
there        0.571575

Weighted combination (w_info=1.0, w_freq=0.0):
       Combined Score  Information Gain  Word Frequency
word                                                   
tares         12946.0          0.953709        0.000058
lares         12945.0          0.943887        0.000054
cares         12944.0          0.935361        0.003878
pares         12943.0          0.932017        0.000111
dares         12942.0          0.931812        0.000429

Recommended word:
tares

00000

856 valid words.

Frequent letters in common positions):
       Information Gain
doily          0.553119
noily          0.546249
colin          0.534056
poind          0.5

8